In [1]:
import yasa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.2)

In [2]:
# Load data
f = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data = f['data']
chan = f['chan']
sf = 100.
times = np.arange(data.shape[-1]) / sf
data.shape

(3, 2161058)

In [3]:
# Apply the detection
sp = yasa.spindles_detect(data, sf, ch_names=chan)
sp.summary().head().round(2)

,Start,Peak,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,522.54,522.78,523.79,1.25,45.51,10.85,2.13,0.47,13.08,16.0,0.19,Cz,0
1,585.51,586.09,586.38,0.87,59.44,13.19,2.26,0.47,12.95,11.0,0.66,Cz,0
2,598.06,598.62,599.54,1.48,79.92,15.98,2.37,0.42,12.89,19.0,0.38,Cz,0
3,604.36,604.73,605.12,0.76,60.65,12.52,2.21,0.35,12.65,9.0,0.48,Cz,0
4,607.53,607.98,608.05,0.52,50.21,13.85,2.36,0.26,13.45,6.0,0.85,Cz,0


In [4]:
%matplotlib widget
import ipywidgets as ipy

# Define mask
mask = sp.get_mask()
spindles_highlight = data * mask
spindles_highlight = np.where(spindles_highlight == 0, np.nan, spindles_highlight)

win_size = 10
n_epochs = int((data.shape[-1] / sf) / win_size)

# Define xlim and xrange
xlim = [0, win_size]
xrange = np.arange(xlim[0] * sf, (xlim[1] * sf + 1), dtype=int)

# Plot
fig, ax = plt.subplots(figsize=(12, 4))
plt.plot(times[xrange], data[0, xrange], 'k', lw=1)
plt.plot(times[xrange], spindles_highlight[0, xrange], 'indianred')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (uV)')
fig.canvas.header_visible = False
fig.tight_layout()


# WIDGETS
layout = ipy.Layout(
    width="50%",
    justify_content='center', 
    align_items='center'
)


sl_ep = ipy.IntSlider(
    min=0, 
    max=n_epochs,
    step=1, 
    value=0, 
    layout=layout,
    description="Epoch:",
)

sl_amp = ipy.IntSlider(
    min=50, 
    max=500,
    step=25, 
    value=200,
    layout=layout,
    orientation='horizontal',
    description="Amplitude:"
)

dd_ch = ipy.Dropdown(options=chan,
    value=chan[0],
    description='Channel:'
)

dd_win = ipy.Dropdown(
    options=[1, 5, 10, 30, 60],
    value=win_size,
    description='Window size:',
)
 
def update(epoch, amplitude, channel, win_size):
    """Update plot."""
    n_epochs = int((data.shape[-1] / sf) / win_size)
    sl_ep.max = n_epochs
    xlim = [epoch * win_size, (epoch + 1) * win_size]
    xrange = np.arange(xlim[0] * sf, (xlim[1] * sf), dtype=int)
    try:
        ax.lines[0].set_data(times[xrange], data[dd_ch.index, xrange])
        ax.lines[1].set_data(times[xrange], spindles_highlight[dd_ch.index, xrange])
        ax.set_xlim(xlim)
    except IndexError:
        pass
    ax.set_ylim([-amplitude, amplitude])

ipy.interact(update, epoch=sl_ep, amplitude=sl_amp, channel=dd_ch, win_size=dd_win);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/raphael/.pyenv/versions/3.8.3/lib/python3.8/site-packages/traitlets/traitlets.py:567: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


interactive(children=(IntSlider(value=0, description='Epoch:', layout=Layout(align_items='center', justify_con…